In [ ]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
train = pd.read_csv("trainYX.csv")
test = pd.read_csv("testX.csv")
train = train.drop("id", axis=1)
test = test.drop("id", axis = 1)
train

,y,1,2,3,4,5,6,7,8,9,...,1015,1016,1017,1018,1019,1020,1021,1022,1023,1024
0,3,227,233,237,235,233,233,230,226,232,...,240,234,237,227,233,240,230,223,227,229
1,8,232,250,252,238,228,238,244,241,252,...,254,253,243,245,240,216,243,247,251,237
2,3,223,223,237,247,225,238,226,234,246,...,213,240,230,241,229,240,237,219,233,226
3,8,180,215,228,218,217,223,228,235,213,...,221,214,243,224,216,222,230,233,214,182
4,8,193,202,214,224,232,239,246,250,249,...,245,247,248,248,242,231,221,214,203,194
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,8,233,233,233,232,231,229,228,227,233,...,232,233,227,225,224,225,229,233,236,238
4996,3,239,239,237,236,235,234,234,234,234,...,233,231,232,233,234,235,235,235,235,234
4997,3,237,227,225,235,238,230,226,231,231,...,232,235,230,226,209,226,230,223,236,219
4998,3,241,242,243,241,239,238,238,240,242,...,244,242,240,240,239,238,238,239,239,239


In [ ]:
print(train.shape)
print(test.shape)

(5000, 1025)
(5000, 1024)


In [ ]:
train_y = train['y'].astype('int32')
train_x = train.drop(['y'], axis = 1).astype('int32')
test_x = test.astype('int32')
train_x.shape, train_y.shape

((5000, 1024), (5000,))

In [ ]:
train_x = train_x.values.reshape(-1, 32, 32, 1)
train_x = train_x / 255
test_x = test_x.values.reshape(-1, 32, 32, 1)
test_x = test_x / 255
train_x.shape, test_x.shape

((5000, 32, 32, 1), (5000, 32, 32, 1))

In [ ]:
train_y

0       3
1       8
2       3
3       8
4       8
       ..
4995    8
4996    3
4997    3
4998    3
4999    3
Name: y, Length: 5000, dtype: int32

In [ ]:
train_y = tf.keras.utils.to_categorical(train_y, 10)
train_y.shape

(5000, 10)

In [ ]:
print(train['y'].head())

0    3
1    8
2    3
3    8
4    8
Name: y, dtype: int64


In [ ]:
print(train_y[0:5,:])

[[0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]]


In [ ]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation= 'relu', input_shape=(32,32,1)),
    tf.keras.layers.Conv2D(32, (3, 3), activation= 'relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(64, (3, 3), activation= 'relu', padding = 'Same'),
    tf.keras.layers.Conv2D(64, (3, 3), activation= 'relu', padding = 'Same'),
    tf.keras.layers.MaxPooling2D(pool_size= (2, 2), strides = (2, 2)),
    tf.keras.layers.Dropout(0.25),
    tf.keras.layers.Conv2D(64, (3, 3), activation= 'relu', padding = 'Same'),
    tf.keras.layers.Conv2D(64, (3, 3), activation= 'relu', padding = 'Same'),
    tf.keras.layers.MaxPooling2D(pool_size= (2, 2), strides = (2, 2)),
    tf.keras.layers.Dropout(0.25),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(256, activation = 'relu'),
    tf.keras.layers.Dense(256, activation = 'relu'),
    tf.keras.layers.Dropout(0.50),
    tf.keras.layers.Dense(10, activation = 'softmax'),
])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 30, 30, 32)        320       
                                                                 
 conv2d_1 (Conv2D)           (None, 28, 28, 32)        9248      
                                                                 
 max_pooling2d (MaxPooling2  (None, 14, 14, 32)        0         
 D)                                                              
                                                                 
 conv2d_2 (Conv2D)           (None, 14, 14, 64)        18496     
                                                                 
 conv2d_3 (Conv2D)           (None, 14, 14, 64)        36928     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 7, 7, 64)          0         
 g2D)                                                   

In [ ]:
class myCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
      if (logs.get('accuracy') > 0.9999):
        print("\nReached 99.9%")
        self.model.stop_training = True

callbacks = myCallback()

In [ ]:
Optimizer = tf.keras.optimizers.Adam(
    learning_rate = 0.0005,
    beta_1 = 0.9,
    beta_2 = 0.999,
    epsilon = 1e-07,
    name = 'Adam'
)


In [ ]:
%%time
model.compile(optimizer=Optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(train_x, train_y, batch_size = 50, epochs = 25, callbacks=[callbacks])

Epoch 1/24
100/100 [==============================] - 2s 9ms/step - loss: 0.0089 - accuracy: 0.9984
Epoch 2/24
100/100 [==============================] - 1s 7ms/step - loss: 0.0064 - accuracy: 0.9972
Epoch 3/24
100/100 [==============================] - 1s 6ms/step - loss: 0.0028 - accuracy: 0.9990
Epoch 4/24
100/100 [==============================] - 1s 6ms/step - loss: 0.0054 - accuracy: 0.9986
Epoch 5/24
100/100 [==============================] - 1s 6ms/step - loss: 0.0130 - accuracy: 0.9976
Epoch 6/24
100/100 [==============================] - 1s 6ms/step - loss: 0.0059 - accuracy: 0.9980
Epoch 7/24
100/100 [==============================] - 1s 6ms/step - loss: 0.0025 - accuracy: 0.9992
Epoch 8/24
100/100 [==============================] - 1s 6ms/step - loss: 0.0038 - accuracy: 0.9988
Epoch 9/24
100/100 [==============================] - 1s 6ms/step - loss: 0.0131 - accuracy: 0.9978
Epoch 10/24
100/100 [==============================] - 1s 7ms/step - loss: 0.0038 - accuracy: 0.9988

In [ ]:
results = model.predict(test_x)
results = np.argmax(results, axis = 1)
results = pd.Series(results, name="y")

157/157 [==============================] - 0s 2ms/step


In [ ]:
results

0       8
1       8
2       3
3       8
4       8
       ..
4995    8
4996    8
4997    8
4998    3
4999    8
Name: y, Length: 5000, dtype: int64

In [ ]:
submission = pd.concat([pd.Series(range(5000, 10000), name = "id"), results], axis = 1)
submission.to_csv("SampleSubmission6.csv", index=False)